In [ ]:
conda install tinydb

In [1]:
#Erstellen einer Datenbank

from tinydb import TinyDB
db = TinyDB('db_Inventar.json')

In [2]:
# JSON-Dokument einfügen
db.insert({
    "stadt": "Mainz",
    "strassen": [
        {
            "name": "Leichhof",
            "hausnummern": [
                {
                    "nummer": "12",
                    "denkmale": [
                        {
                            "typ": "Haus",
                            "beschreibung": "Gotisches Bürgerhaus aus dem 15. Jahrhundert",
                            "material": "Sandstein",
                            "epoche": "Gotik",
                            "kuenstler": "Unbekannt",
                            "besonderheiten": "Spitzbogenfenster mit Maßwerk"
                        }
                    ]
                },
                {
                    "nummer": "23",
                    "denkmale": [
                        {
                            "typ": "Kunstwerk",
                            "beschreibung": "Barockes Wandrelief",
                            "material": "Marmor",
                            "epoche": "Barock",
                            "kuenstler": "Peter Paul Rubens",
                            "besonderheiten": "Detailreiche Darstellung eines Engels"
                        }
                    ]
                }
            ]
        }
    ]
})

1

In [4]:
# Abfrage Nr. 1

from tinydb import Query

# Alle Daten in der Datenbank anzeigen
print(db.all())

[{'stadt': 'Mainz', 'strassen': [{'name': 'Leichhof', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [5]:
# Abfrage Nr. 2

result = db.search(Query().strassen.any(lambda x: x['name'] == "Leichhof"))
print(result)


[{'stadt': 'Mainz', 'strassen': [{'name': 'Leichhof', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [10]:
# Alternative zu Abfrage Nr. 2
def is_leichhof(street):
    return street['name'] == "Leichhof"

result = db.search(Query().strassen.any(is_leichhof))
print(result)


[{'stadt': 'Mainz', 'strassen': [{'name': 'Leichhof', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [8]:
# Abfrage Nr. 3

result = [
    haus for entry in db.search(Query().strassen.any(
        lambda strasse: strasse['name'] == "Leichhof"
    ))
    for strasse in entry['strassen'] if strasse['name'] == "Leichhof"
    for haus in strasse['hausnummern'] if haus['nummer'] == "12"
]

print(result)



[{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}]


In [ ]:
# Beispiel für das 'write_back' in früheren Versionen !ACHTUNG! nicht mehr aktuell !ACHTUNG!
result = db.search(Query().strassen.any(lambda x: x['name'] == "Leichhof"))
for item in result:
    item['strassen'][0]['name'] = "Neuer Name"
db.write_back(result)


In [15]:
# Statt write_back können wir mit update arbeiten

from tinydb import TinyDB, Query

# Erstelle eine Datenbankinstanz
db = TinyDB('db_Inventar.json')

# Abfrage nach einem bestimmten Dokument
Stadt = Query()
result = db.search(Stadt.strassen.any(lambda x: x['name'] == "Leichhof"))

# Angenommen, du möchtest den Namen der Straße ändern
for item in result:
    # Bearbeite das Dokument direkt
    item['strassen'][0]['name'] = "Neuer Name"

    # Aktualisiere das Dokument in der Datenbank
    db.update(item, Stadt.stadt == "Mainz")


In [16]:
print(db.all())

[{'stadt': 'Mainz', 'strassen': [{'name': 'Neuer Name', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [2]:
# Einträge löschen
# Löschen der Straße "Neuer Name"

db.remove(Query().strassen.any(lambda street: street['name'] == "Neuer Name"))


[1]

In [3]:
print(db.all())

[]


In [5]:
# Einzelnen Wert löschen durch "update"-Befehl
from tinydb import TinyDB, Query

# Initialisiere die Datenbank
db = TinyDB('db_Inventar.json')

# Suche den Wert "Sandstein" im Dokument und lösche ihn
db.update(
    lambda doc: [
        denkmal.update({'material': None})  # Setze den Wert auf None
        for strasse in doc['strassen']
        for hausnummer in strasse['hausnummern']
        for denkmal in hausnummer['denkmale']
        if denkmal.get('material') == "Sandstein"  # Prüfe, ob der Wert "Sandstein" ist
    ],
)


[1]

In [13]:
print(db.all())

[{'stadt': 'Mainz', 'strassen': [{'name': 'Leichhof', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': None, 'epoche': 'Gotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [16]:
# Einzelne Werte überschreiben mit Update
from tinydb import TinyDB, Query

db = TinyDB('db_Inventar.json')

# Suche den Wert None und überschreibe ihn mit Sandstein
db.update(
    lambda doc: [
        denkmal.update({'material': "Sandstein"})  
        for strasse in doc['strassen']
        for hausnummer in strasse['hausnummern']
        for denkmal in hausnummer['denkmale']
        if denkmal.get('material') == None  # Prüfe, ob der Wert "None" ist
    ],
)

[1]

In [18]:
# Aktualisiere die Epoche für Hausnummer 12 in der Straße Leichhof
db.update(
    lambda doc: [
        denkmal.update({'epoche': 'Neogotik'})  # Ersetze 'Gotik' durch 'Neogotik'
        for strasse in doc['strassen']
        if strasse['name'] == "Leichhof"  # Prüfe, ob es die Straße 'Leichhof' ist
        for hausnummer in strasse['hausnummern']
        if hausnummer['nummer'] == "12"  # Prüfe, ob es die Hausnummer '12' ist
        for denkmal in hausnummer['denkmale']
        if denkmal.get('epoche') == "Gotik"  # Prüfe, ob die Epoche 'Gotik' ist
    ]
)


[1]

In [24]:
# Neues Denkmal zur bestehenden Hausnummer 12 hinzufügen

from tinydb import TinyDB, Query

db = TinyDB('db_Inventar.json')

# Definiere das neue Denkmal, das hinzugefügt werden soll
neues_denkmal = {
    "typ": "Skulptur",
    "beschreibung": "Moderne Skulptur eines Adlers",
    "material": "Bronze",
    "epoche": "Moderne",
    "kuenstler": "Max Mustermann",
    "besonderheiten": "Flügelspannweite von 2 Metern"
}

# Suche nach der Straße "Leichhof" und Hausnummer "12"
strasse = Query()

result = db.search(strasse.strassen.any(
    lambda x: x['name'] == "Leichhof" and any(h['nummer'] == "12" for h in x['hausnummern'])
))

# Wenn das Dokument existiert, füge das neue Denkmal hinzu
if result:
    for strasse_entry in result:
        for hausnummer_entry in strasse_entry['strassen']:
            for haus in hausnummer_entry['hausnummern']:
                if haus['nummer'] == "12":
                    haus['denkmale'].append(neues_denkmal)  # Füge das neue Denkmal hinzu

    # Das geänderte Dokument in der Datenbank speichern
    db.update(result[0], strasse.strassen.any(
        lambda x: x['name'] == "Leichhof" and any(h['nummer'] == "12" for h in x['hausnummern'])
    ))

print(db.all())


[{'stadt': 'Mainz', 'strassen': [{'name': 'Leichhof', 'hausnummern': [{'nummer': '12', 'denkmale': [{'typ': 'Haus', 'beschreibung': 'Gotisches Bürgerhaus aus dem 15. Jahrhundert', 'material': 'Sandstein', 'epoche': 'Neogotik', 'kuenstler': 'Unbekannt', 'besonderheiten': 'Spitzbogenfenster mit Maßwerk'}, {'typ': 'Skulptur', 'beschreibung': 'Moderne Skulptur eines Adlers', 'material': 'Bronze', 'epoche': 'Moderne', 'kuenstler': 'Max Mustermann', 'besonderheiten': 'Flügelspannweite von 2 Metern'}]}, {'nummer': '23', 'denkmale': [{'typ': 'Kunstwerk', 'beschreibung': 'Barockes Wandrelief', 'material': 'Marmor', 'epoche': 'Barock', 'kuenstler': 'Peter Paul Rubens', 'besonderheiten': 'Detailreiche Darstellung eines Engels'}]}]}]}]


In [28]:
# Ein Denkmal zu neuee Hausnummer zu einer bestehenden Straße hinzufügen
from tinydb import TinyDB, Query

db = TinyDB('db_Inventar.json')

neues_denkmal = {
    "typ": "Gedenktafel",
    "beschreibung": "Gedenktafel für die Geschichte der Straße",
    "material": "Bronze",
    "epoche": "Moderne",
    "kuenstler": "Künstlername",
    "besonderheiten": "Eingerahmt und an der Wand montiert"
}

# Definiere die neue Hausnummer mit dem Denkmal
neue_hausnummer = {
    "nummer": "24",
    "denkmale": [neues_denkmal]  # Das Denkmal wird als Liste hinzugefügt
}

# Suche nach der Straße "Leichhof"
strasse = Query()

result = db.search(strasse.strassen.any(
    lambda x: x['name'] == "Leichhof"
))

# Wenn das Dokument existiert, füge die neue Hausnummer und das Denkmal hinzu
if result:
    for strasse_entry in result:
        strasse_entry['strassen'][0]['hausnummern'].append(neue_hausnummer)

    # Aktualisiere das Dokument in der Datenbank mit der neuen Hausnummer und dem Denkmal
    db.update(result[0], strasse.strassen.any(
        lambda x: x['name'] == "Leichhof"
    ))


In [29]:
# Beispiel für das Hinzufügen einer neuen Straße mit Hausnummern und Denkmälern
db.insert({
    "stadt": "Mainz",
    "strassen": [
        {
            "name": "Neue Straße",
            "hausnummern": [
                {
                    "nummer": "1",
                    "denkmale": [
                        {
                            "typ": "Haus",
                            "beschreibung": "Neues Gebäude im modernen Stil",
                            "material": "Beton",
                            "epoche": "Moderne",
                            "kuenstler": "Max Mustermann",
                            "besonderheiten": "Fassadenverkleidung aus Glas"
                        }
                    ]
                }
            ]
        }
    ]
})


2